In [1]:
import pandas as pd

In [15]:
questions = pd.read_csv('Data/questions.csv')

questions.head()

,Unnamed: 0,questiontext,questionid
0,0,What is your age?,1
1,1,What is your gender?,2
2,2,What country do you live in?,3
3,3,"If you live in the United States, which state ...",4
4,4,Are you self-employed?,5


In [21]:
answers = pd.read_csv('Data/answers.csv')

answers.head()

,Unnamed: 0,AnswerText,SurveyID,UserID,QuestionID
0,0,37,2014,1,1
1,1,44,2014,2,1
2,2,32,2014,3,1
3,3,31,2014,4,1
4,4,31,2014,5,1


In [38]:
pd.merge(questions , answers , left_on='questionid', right_on='QuestionID')



,Unnamed: 0_x,questiontext,questionid,Unnamed: 0_y,AnswerText,SurveyID,UserID,QuestionID
0,1,What is your gender?,1,0,37,2014,1,1
1,1,What is your gender?,1,1,44,2014,2,1
2,1,What is your gender?,1,2,32,2014,3,1
3,1,What is your gender?,1,3,31,2014,4,1
4,1,What is your gender?,1,4,31,2014,5,1
...,...,...,...,...,...,...,...,...
214573,104,Do you work remotely?,104,98673,-1,2016,2689,104
214574,104,Do you work remotely?,104,98674,-1,2016,2690,104
214575,104,Do you work remotely?,104,98675,Maybe,2016,2691,104
214576,104,Do you work remotely?,104,98676,Maybe,2016,2692,104


1) Look in column to make sure answers are standardized 
    Check and see if you need either remove or change a record to make sure they are all the same 
2) Keep only questions we need for our analysis 
3) Merge it with answer csv
        Try to get questions as columns and answer as rows for each survey taker
4) check for duplicates and missing values 
5) Check column names. 
        Captialize words 
6) 
        

,Unnamed: 0,questiontext,questionid
0,0,What is your age?,1
1,1,What is your gender?,2
2,2,What country do you live in?,3
3,3,"If you live in the United States, which state ...",4
4,4,Are you self-employed?,5
...,...,...,...
100,100,Do you think that team members/co-workers woul...,114
101,101,"If yes, what condition(s) have you been diagno...",115
102,102,"If maybe, what condition(s) do you believe you...",116
103,103,Which of the following best describes your wor...,117
